# CNN Model Creation

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import Input, activations
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.losses import CategoricalCrossentropy
from keras.models import Sequential
from keras.datasets import cifar10
from sklearn.preprocessing import LabelBinarizer

In [7]:
# Data Preparation and Processing
data = cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

#Target transformation
output_encoder = LabelBinarizer()
y_train = output_encoder.fit_transform(y_train)
y_test = output_encoder.transform(y_test)

 # Shapes of data variables
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_test shape: ", X_test.shape)
print("y_test shape: ", y_test.shape)

/Users/justinbell/Desktop/Computer Science/Projects/ML Models/Object Classifier/.venv/lib/python3.12/site-packages/keras/src/datasets/cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


X_train shape:  (50000, 32, 32, 3)
y_train shape:  (50000, 10)
X_test shape:  (10000, 32, 32, 3)
y_test shape:  (10000, 10)


In [8]:
print(output_encoder.classes_)

[0 1 2 3 4 5 6 7 8 9]


In [9]:
X_train.shape[1:4]

(32, 32, 3)

In [10]:
print(X_train[1])

[[[0.60392157 0.69411765 0.73333333]
  [0.49411765 0.5372549  0.53333333]
  [0.41176471 0.40784314 0.37254902]
  ...
  [0.35686275 0.37254902 0.27843137]
  [0.34117647 0.35294118 0.27843137]
  [0.30980392 0.31764706 0.2745098 ]]

 [[0.54901961 0.62745098 0.6627451 ]
  [0.56862745 0.6        0.60392157]
  [0.49019608 0.49019608 0.4627451 ]
  ...
  [0.37647059 0.38823529 0.30588235]
  [0.30196078 0.31372549 0.24313725]
  [0.27843137 0.28627451 0.23921569]]

 [[0.54901961 0.60784314 0.64313725]
  [0.54509804 0.57254902 0.58431373]
  [0.45098039 0.45098039 0.43921569]
  ...
  [0.30980392 0.32156863 0.25098039]
  [0.26666667 0.2745098  0.21568627]
  [0.2627451  0.27058824 0.21568627]]

 ...

 [[0.68627451 0.65490196 0.65098039]
  [0.61176471 0.60392157 0.62745098]
  [0.60392157 0.62745098 0.66666667]
  ...
  [0.16470588 0.13333333 0.14117647]
  [0.23921569 0.20784314 0.22352941]
  [0.36470588 0.3254902  0.35686275]]

 [[0.64705882 0.60392157 0.50196078]
  [0.61176471 0.59607843 0.50980392]


In [19]:
callbacks = []
epochs = 20
batch_size = 100
checkpoint = ModelCheckpoint(f"conv_weights/Conv2D Model/{epochs}e-{batch_size}bs" + "-{loss:.4f}" + ".keras", monitor='loss', verbose=1,
                                         save_best_only=True, mode='min')
callbacks = [checkpoint]

In [25]:
#Model Creation

model = Sequential([
    Conv2D(filters=32, kernel_size=(5,5), activation=activations.relu),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(filters=48, kernel_size=(7,7), activation=activations.relu),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(filters=100, kernel_size=(5,5), activation=activations.relu, padding='same'),
    Flatten(),
    Dense(units=50, activation=activations.relu),
    Dropout(0.5),
    Dense(units=10, activation=activations.softmax)
])
model.build(input_shape=(None,) + X_train.shape[1:4])
print(model.summary())

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 32)     │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 8, 8, 48)       │        75,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 4, 4, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 4, 4, 100)      │       120,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 50)             │        80,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 278,404 (1.06 MB)

 Trainable params: 278,404 (1.06 MB)

 Non-trainable params: 0 (0.00 B)

None


In [26]:
model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=['accuracy'])
# model.load_weights("conv_weights/Conv2D Model 3/20e-100bs-0.1959.keras")
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=False, callbacks=callbacks, verbose=1, validation_split=0.1)

Epoch 1/20
449/450 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.2184 - loss: 2.0682
Epoch 1: loss did not improve from 0.52767
450/450 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.2973 - loss: 1.8810 - val_accuracy: 0.4390 - val_loss: 1.5437
Epoch 2/20
448/450 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.4415 - loss: 1.5540
Epoch 2: loss did not improve from 0.52767
450/450 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 0.4609 - loss: 1.5084 - val_accuracy: 0.5460 - val_loss: 1.2854
Epoch 3/20
450/450 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.5068 - loss: 1.3956
Epoch 3: loss did not improve from 0.52767
450/450 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.5178 - loss: 1.3710 - val_accuracy: 0.5858 - val_loss: 1.2103
Epoch 4/20
448/450 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.5487 - loss: 1.2890
Epoch 4: loss did not improve from 0.52767
450/450 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.5578 - loss: 1.2652 - val_accuracy: 0.6092 - val_loss: 1.1200
Epoch

In [ ]:
predictions = model.predict(X_train[:10])
pred_decode = output_encoder.inverse_transform(predictions)
print(pred_decode)
print(output_encoder.inverse_transform(y_train[:10]))

In [ ]:
# import tensorflow as tf
# converter = tf.lite.TFLiteConverter.from_keras_model(model) # or .from_saved_model(saved_model_dir)
# tflite_model = converter.convert()
# with open('model.tflite', 'wb') as f:
#     f.write(tflite_model)

Saved artifact at '/var/folders/0_/nmh5w64x3q5_ffwfkr35sscw0000gn/T/tmphi4ywg6h'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_43')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  4895227984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895229136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895229712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895229904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230864: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895230672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4895231248: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1766688857.598078  212361 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1766688857.598484  212361 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
I0000 00:00:1766688857.608609  212361 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled


In [20]:
# Save the model to a .h5 file
model.save("object_classifier_model4.keras")

In [33]:
model.output_names = ['output']

In [ ]:
import coremltools as ct


# # Convert the model
# mlmodel = ct.convert(
#     'object_classifier_model4.h5',
#     convert_to="mlprogram", # Use mlprogram for better performance on newer iOS versions
#     minimum_deployment_target=ct.target.iOS15, # Specify your minimum iOS target
#     source="tensorflow"
# )

image_input = ct.converters.mil.input_types.ImageType(
    name='input_image', 
    shape=(1, 3, 224, 224), 
    scale=1.0/255.0, 
    bias=[0.0, 0.0, 0.0], 
    color_layout='RGB'
)

mlmodel = ct.converters.convert(model=model, 
                     inputs=[image_input], 
                     source="auto", convert_to="mlprogram", 
                     minimum_deployment_target=ct.target.iOS16)

# mlmodel = ct.convert(model=model, 
#                      inputs=[ct.TensorType(dtype=np.float16)], 
#                      source="tensorflow", convert_to="mlprogram", 
#                      minimum_deployment_target=ct.target.iOS16,
#                      output_names=['output'])
# Save the converted model
mlmodel.save("ObjectClassifierModel.mlpackage")